In [1]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob 
import contractions

> - The dataset is publicly available at on Kaggle at:
https://www.kaggle.com/efehandanisman/skytrax-airline-reviews,

In [2]:
df = pd.read_excel('Airline_Review_Data/All_Reviews.xlsx')
df

,airline,overall,author,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,Adria Airways,8.0,M Jager,12th October 2018,âœ… Trip Verified | Ljubljana to Munich. The h...,NaN,Family Leisure,Economy Class,Ljubljana to Munich,October 2018,4.0,4.0,3.0,NaN,5.0,5.0,yes
1,Adria Airways,1.0,Giulia Rossi,5th October 2018,Not Verified | Zurich to Ljubljana. Very poor ...,NaN,Business,Economy Class,Zurich to Ljubljana,October 2018,2.0,1.0,NaN,1.0,1.0,1.0,no
2,Adria Airways,1.0,Galya Slavov,29th July 2018,âœ… Trip Verified | Vienna to Sofia. The fligh...,NaN,Family Leisure,Economy Class,Vienna to Sofia,July 2018,4.0,1.0,1.0,NaN,4.0,1.0,no
3,Adria Airways,2.0,Loic Jouan,19th July 2018,âœ… Trip Verified | We were traveling from Par...,NaN,Solo Leisure,Economy Class,Paris to Skopje via Ljubljana,2018-05-01 00:00:00,3.0,3.0,NaN,NaN,3.0,2.0,no
4,Adria Airways,2.0,P Gamirj,30th June 2018,âœ… Trip Verified | Ljubljana to Munich. Adria...,NaN,Business,Economy Class,Ljubljana to Munich,June 2018,1.0,2.0,2.0,NaN,2.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65942,WOW air,10.0,Mike Causer,21st March 2013,Flew London Gatwick - Reykjavik return on 15/1...,NaN,NaN,Economy Class,NaN,NaN,4.0,4.0,4.0,NaN,NaN,5.0,yes
65943,WOW air,6.0,Halldorsson Sigmundur,24th January 2013,Flew with this new entrant to the Icelandic ma...,NaN,NaN,Economy Class,NaN,NaN,4.0,4.0,NaN,NaN,NaN,3.0,yes
65944,WOW air,9.0,N Holdsworth,10th January 2013,Second time on WOW LGW to KEF 21st December. S...,NaN,NaN,Economy Class,NaN,NaN,5.0,5.0,5.0,NaN,NaN,4.0,yes
65945,WOW air,9.0,Michael Shoesmith,23rd December 2012,7th December 2012 LGW-KEF. Iceland Express and...,NaN,NaN,Economy Class,NaN,NaN,4.0,3.0,4.0,NaN,NaN,4.0,yes


# Exploratory data analysis

In [3]:
df['airline'].unique()

array(['Adria Airways', 'Aegean Airlines', 'Aer Lingus',
       'Aeroflot Russian Airlines', 'Aeromexico', 'Air Arabia',
       'Air Canada', 'Air Canada rouge', 'Air China', 'Air Europa',
       'Air France', 'Air India', 'Air New Zealand', 'AirAsia',
       'airBaltic', 'Alaska Airlines', 'Alitalia', 'American Airlines',
       'ANA All Nippon Airways', 'Asiana Airlines', 'Austrian Airlines',
       'Avianca', 'Bangkok Airways', 'British Airways',
       'Brussels Airlines', 'Cathay Pacific Airways',
       'China Eastern Airlines', 'China Southern Airlines',
       'Copa Airlines', 'Delta Air Lines', 'easyJet', 'Egyptair',
       'Emirates', 'Ethiopian Airlines', 'Etihad Airways', 'Eurowings',
       'EVA Air', 'Finnair', 'flydubai', 'Frontier Airlines',
       'Garuda Indonesia', 'Germanwings', 'Gulf Air', 'Iberia',
       'Icelandair', 'IndiGo', 'Jetblue Airways',
       'KLM Royal Dutch Airlines', 'Korean Air', 'Kuwait Airways',
       'LATAM Airlines', 'LOT Polish Airlines', 'Lu

# Data Pre-processing

In [4]:
## Review Data Cleaning
df['customer_review'] = df['customer_review'].str.replace('"', "'",regex=True)
df['customer_review'] = df['customer_review'].str.replace(r"^âœ… Trip Verified \| ", "",regex=True)
df['customer_review'] = df['customer_review'].str.replace(r"^âœ… Verified Review \| ", "",regex=True)
df['customer_review'] = df['customer_review'].str.replace(r"^Not Verified \| ", "",regex=True)
df['customer_review'] = df['customer_review'].str.replace(r"â€™", "'",regex=True)

In [5]:
# adding one new column
df['review_id'] = df.index
# cast date datatype
df['review_date'] = pd.to_datetime(df['review_date']).dt.date

# Data enrichment

In [6]:
# Adding sentiment label for reviews

# Define sentiment lebeling function to map to dataframe
def sentiment_labeler(text):
    #expanding contraction in text
    expand_text = contractions.fix(text)
    # remove non-ascii character in text
    ascii_text = ("".join(c for c in expand_text if ord(c)<128))
    score = TextBlob(ascii_text).sentiment.polarity
    if score > 0:
        return 'positive'
    else: return 'negative'

df['sentiment_label'] = df['customer_review'].apply(sentiment_labeler)

In [7]:
# adding length of reviews
def term_counter(text):
    expand_text = contractions.fix(text)
    ascii_text = ("".join(c for c in expand_text if ord(c)<128))
    return len(ascii_text.split())
df['review_length'] = df['customer_review'].apply(term_counter)


# Exploratory Data Analysis

In [8]:
# Missing value rate
df.isnull().mean()*100

airline             0.000000
overall             2.926593
author              0.000000
review_date         0.000000
customer_review     0.000000
aircraft           70.100232
traveller_type     39.716742
cabin               4.009280
route              39.760717
date_flown         39.901739
seat_comfort        7.985200
cabin_service       7.933644
food_bev           20.226849
entertainment      32.987096
ground_service     40.318741
value_for_money     2.990280
recommended         2.285168
review_id           0.000000
sentiment_label     0.000000
review_length       0.000000
dtype: float64

> - We can see that the attributes with the highest missing values are aircraft, travel_type,
route, date_flown, ground service, and country. Therefore, we will not incorporate
those data fields into our analysis
> - For numerical attributes that are not fully populated by
figures, when calculating aggregation metrics, we will exclude the missing values. What is good
about this dataset is that the attributes of customer_review and airline, target attributes for
our LDA topic analysis, are fully populated.

In [11]:
period = []
review_range = []
n_reviews =[]
negative_rate = []
seat_comfort = []
cabin_service = []
food_bev = []
inf_entertainment =[]
value_for_money =[]
overall = []
recommendation_rate = []

# Aggregating Data for each ailines:
airline_list = df['airline'].unique()
for airline in airline_list:
    df_airline = df.loc[df['airline'] == airline]
    period.append((df_airline['review_date'].min().strftime('%d-%m-%Y'), df_airline['review_date'].max().strftime('%d-%m-%Y')))
    review_range.append((df_airline['review_length'].min() , df_airline['review_length'].max()))
    n_reviews.append(len(df_airline['customer_review']))
    neg_rate = len(df_airline.loc[df_airline['sentiment_label'] == 'negative'])/len(df_airline)
    negative_rate.append(neg_rate)
    seat_comfort.append(df_airline['seat_comfort'].mean(skipna=True))
    cabin_service.append(df_airline['cabin_service'].mean(skipna=True))
    food_bev.append(df_airline['food_bev'].mean(skipna=True))
    inf_entertainment.append(df_airline['entertainment'].mean(skipna=True))
    value_for_money.append(df_airline['value_for_money'].mean(skipna=True))
    overall.append(df_airline['overall'].mean(skipna=True))
    recommendation_rate.append(df_airline['recommended'].value_counts(normalize=True)['yes'])

aggregation_dict = {'airline' : airline_list,
                'period' : period,
                'review_range' : review_range,
                'n_reviews' : n_reviews,
                'negative_rate' : negative_rate,
                'seat_comfort' : seat_comfort,
                'cabin_service' : cabin_service,
                'food_bev' : food_bev,
                'inf_entertainment' : inf_entertainment,
                'value_for_money' : value_for_money,
                'overall_rating' : overall,
                'recommendation_rate': recommendation_rate
               }
aggregation_df = pd.DataFrame(aggregation_dict)

#adding business model of airlines in the aggregation
business_model = pd.read_csv('Airline_Review_Data/business_model.csv')
business_model['airline'] = business_model['airline'].apply(lambda x : x.strip())
aggregation_df = aggregation_df.merge(business_model[['airline','business_model']], how='left', on='airline')

#re-arrang the columns
aggregation_df = aggregation_df[['airline','business_model', 'period', 'review_range', 'n_reviews',
                                 'negative_rate','overall_rating','recommendation_rate','seat_comfort', 'cabin_service',
                                 'food_bev', 'inf_entertainment','value_for_money']]

In [12]:
# Top airlines with highest negative review rate
aggregation_df[['airline','business_model','review_range','n_reviews',
                'negative_rate','overall_rating',
                'recommendation_rate']].sort_values(by=['negative_rate'], ascending=False).head(13)

,airline,business_model,review_range,n_reviews,negative_rate,overall_rating,recommendation_rate
39,Frontier Airlines,LCC,"(2, 691)",1624,0.482143,2.370014,0.139011
38,flydubai,LCC,"(10, 670)",201,0.467662,3.781250,0.305000
66,Spirit Airlines,LCC,"(8, 730)",2934,0.465235,2.617869,0.179094
76,United Airlines,FSC,"(6, 655)",2829,0.450336,3.029349,0.214079
78,Vueling Airlines,LCC,"(9, 659)",965,0.440415,3.365062,0.256085
9,Air Europa,LCC,"(14, 659)",684,0.438596,4.237624,0.382090
17,American Airlines,FSC,"(5, 699)",2867,0.431810,2.898672,0.201729
7,Air Canada rouge,LCC,"(21, 685)",2192,0.425182,2.998175,0.182239
58,Royal Air Maroc,FSC,"(5, 625)",265,0.422642,4.578475,0.365759
67,Sunwing Airlines,LCC,"(19, 573)",603,0.411277,3.492334,0.242833


> - As we can see, top airlines that are infamous for the dissatisfied remarks from their passenger are
mostly low-cost carriers. Also, the average overall rating is all below the average of 5. And the
negative review rate seems to correlate with the overall rating negatively. Regarding
Recommendation Rate, Air Europa and Royal Air Maroc are even in the list of negativity
receivers; they are the two most recommended airlines among this category.

In [13]:
# Top airlines with lowest negative review rate
aggregation_df[['airline','business_model','review_range','n_reviews',
                'negative_rate','overall_rating',
                'recommendation_rate']].sort_values(by=['negative_rate'], ascending=True).head(15)

,airline,business_model,review_range,n_reviews,negative_rate,overall_rating,recommendation_rate
40,Garuda Indonesia,FSC,"(12, 621)",744,0.071237,8.289367,0.905866
19,Asiana Airlines,FSC,"(5, 525)",447,0.078300,8.060674,0.848276
36,EVA Air,FSC,"(8, 500)",530,0.094340,8.110687,0.855469
56,QantasLink,RC,"(20, 272)",105,0.095238,7.384615,0.786408
27,China Southern Airlines,FSC,"(8, 611)",1722,0.102207,7.774269,0.862049
18,ANA All Nippon Airways,FSC,"(8, 480)",473,0.107822,7.871520,0.835141
57,Qatar Airways,FSC,"(12, 611)",1445,0.110035,7.484765,0.790334
22,Bangkok Airways,RC,"(11, 645)",356,0.112360,7.602817,0.755682
0,Adria Airways,RC,"(23, 391)",85,0.117647,6.191176,0.506024
48,Korean Air,FSC,"(7, 518)",498,0.122490,7.627291,0.772541


> - For top-service airlines, top airlines that have the highest positive review rate from their flyers are
all full service and regional carriers, and the majority of them are Asia-based airlines such as All
Nippon Airways, Cathay Pacific, Thai Airways, Korean Air, China Southern Airlines... 
> - Also, the
average overall ratings are all above the average of 5. And the negative review rate tent to also
has a negative correlation with the overall rating for those top Airline.
However, we do have outlier players in the dataset. 
> - For instance, Virgin America and Indigo are
two low-cost carriers that receive negative review ratings below 20% (18%, 19%
correspondingly). Regarding full-service carriers, United Airlines and American Airlines are
those having negative review rates of 45% and 43%, respectively

# Correlation Analysis

In [14]:
cor_df = aggregation_df[['negative_rate','overall_rating','recommendation_rate','seat_comfort', 'cabin_service',
                                 'food_bev', 'inf_entertainment','value_for_money']]
cor_df.corr()

,negative_rate,overall_rating,recommendation_rate,seat_comfort,cabin_service,food_bev,inf_entertainment,value_for_money
negative_rate,1.000000,-0.951132,-0.938462,-0.909700,-0.950942,-0.878725,-0.672949,-0.921754
overall_rating,-0.951132,1.000000,0.985835,0.950700,0.964335,0.938842,0.710117,0.971644
recommendation_rate,-0.938462,0.985835,1.000000,0.928777,0.957799,0.929060,0.708545,0.972129
seat_comfort,-0.909700,0.950700,0.928777,1.000000,0.909242,0.924805,0.768817,0.941753
cabin_service,-0.950942,0.964335,0.957799,0.909242,1.000000,0.934325,0.732108,0.927584
food_bev,-0.878725,0.938842,0.929060,0.924805,0.934325,1.000000,0.790905,0.918709
inf_entertainment,-0.672949,0.710117,0.708545,0.768817,0.732108,0.790905,1.000000,0.694070
value_for_money,-0.921754,0.971644,0.972129,0.941753,0.927584,0.918709,0.694070,1.000000


> - For top-service airlines, top airlines that have the highest positive review rate from their flyers are
all full service and regional carriers, and the majority of them are Asia-based airlines such as All
Nippon Airways, Cathay Pacific, Thai Airways, Korean Air, China Southern Airlines... 
> - Also, the
average overall ratings are all above the average of 5. And the negative review rate tent to also
has a negative correlation with the overall rating for those top Airline.
However, we do have outlier players in the dataset. 
> - For instance, Virgin America and Indigo are
two low-cost carriers that receive negative review ratings below 20% (18%, 19%
correspondingly). Regarding full-service carriers, United Airlines and American Airlines are
those having negative review rates of 45% and 43%, respectively

# Confusion matrices

In [15]:
#Confusion Matrix of Recommendation and Review Sentiment
pd.crosstab(df['recommended'],df['sentiment_label'])

sentiment_label,negative,positive
recommended,,
no,17618,16276
yes,1374,29172


> - A surprising insight is that for customers who received good service (positive review), over one-third of them are
not likely to recommend the airlines for their friends and families, compare to a normal
assumption that passengers would refer to the Airline with which they had a great flight with and
vice versa.

In [16]:
# Confusion Matrix of Value for Money and Review Sentiment
a= pd.crosstab(df['sentiment_label'],df['value_for_money'])
a['>3'] = a[4] +a[5]
a['<3'] = a[1] +a[2]
a[['<3','>3']].T

sentiment_label,negative,positive
value_for_money,,
<3,14919,12480
>3,1450,26857


> - We can observe that the vast majority of negative feedback reviews think that the service they get
is not worth the price they paid. However, for the section of positive reviewers, we can see a
similar trend, in which over 30% of them do not feel satisfied with the value from the service for
the price of the flights they paid. And this again reaffirms our previous claim of the
recommendation possibility of a positive review segment of the passenger.
> - In conclusion, we could say that passengers, who flight with full-service and regional carriers,
tend to receive a better service compared to the low-cost flyers, but they will not likely to
recommend the carriers if they think that the good service, they get does not worth the money
they spent.